In [291]:
import pandas as pd

data = {
  "qty": [1, 2, 30, 1000],
  'price': [1,2,3,1],
  'lotSize': [1,1,10,1],
  'percentTgt': [.3,.2,.7,0]
}

df = pd.DataFrame(data, index = ["a", "b", "c",'@RUB'])

df

,qty,price,lotSize,percentTgt
a,1,1,1,0.3
b,2,2,1,0.2
c,30,3,10,0.7
@RUB,1000,1,1,0.0


In [292]:
import numpy as np

def stocksToLots(df):
    df['lotQty'] = df['qty'] / df['lotSize']
    df['lotPrice'] = df['price'] * df['lotSize']
    df['value'] = df['lotQty']*df['lotPrice']
    return df

def applyLots(df):
    df['valueChgNoLots'] = df['valueTgt1'] - df['value']
    df['lotQtyChg'] = (df['valueChgNoLots'] / df['lotPrice']).apply(np.round)
    df['valueChg'] = df['lotQtyChg'] * df['lotPrice']
    return df

def renameColumns(df):
    rf = df[['lotPrice','value','valueChg']]
    rf['percentTgt'] = df['percentTgt1'] 
    rf['valueTgt'] = df['valueTgt1'] 
    return rf

def prelimChangeValue(df):
    df['percentTgt1'] = df['percentTgt'] / df['percentTgt'].sum()
    df['valueTgt1'] = df['percentTgt1'] * df['value'].sum()
    df = applyLots(df)

    cashChange = -  df['valueChg'].sum()+ df.at['@RUB','valueChg']
    df.at['@RUB','valueChg'] = cashChange
    df.at['@RUB','lotQtyChg'] = cashChange

    return renameColumns(df)

def adjustTolerance(df,tolerance):
    pd.options.mode.chained_assignment = None  # default='warn'

    df['percentTgt1'] = 0
    df.loc[ abs(df['valueChg']) > tolerance, 'percentTgt1'] = df['percentTgt'] 
    df['percentTgt1'] = df['percentTgt1'] / df['percentTgt1'].sum()

    df['valueTgt1'] = df['percentTgt1'] * df['value'].sum() 
    df = applyLots(df)

    return renameColumns(df)

def adjustRemainder(df):
    maxChgIndex = df[df['valueChg']==df['valueChg'].max()].index[0]
    cashAdj = np.ceil((df['valueChg'].sum() / df.at[maxChgIndex,'lotPrice']))
    df.at[maxChgIndex,'lotQtyChg'] = -cashAdj
    df.at['@RUB','lotQtyChg'] = cashAdj * df.at[maxChgIndex,'lotPrice']
    return df

# def finalChangeValue(df):
#     df = df.fillna(0)
#     df['lotQtyChg'] = df['lotQtyChg2']+df['lotQtyChg3']
#     df['valueChg'] = df['lotQtyChg'] * df['lotPrice']
#     df['qtyChg'] = df['lotQtyChg'] * df['lotSize']
#     return df[['qtyChg','lotQtyChg','valueChg']]

tolerance=200

# use lots instead of single assets
df = stocksToLots(df)

# 1st pass -  no limit to sum of change
df = prelimChangeValue(df)

# # 2nd pass - limit changes by tolerance
df = adjustTolerance(df,tolerance)

# # 3rd pass - correct remainder due to rouding
df = adjustRemainder(df)

# df = finalChangeValue(df)

df

/var/folders/z8/x3vvmmj13bgbk06t2hnjqt_00000gn/T/ipykernel_1103/3979193384.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.25       0.58333333 0.        ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[ abs(df['valueChg']) > tolerance, 'percentTgt1'] = df['percentTgt']


,lotPrice,value,valueChg,percentTgt,valueTgt,lotQtyChg
a,1,1.0,328.0,0.3,328.5,NaN
b,2,4.0,-4.0,0.0,0.0,NaN
c,30,90.0,690.0,0.7,766.5,-1.0
@RUB,1,1000.0,-1000.0,0.0,0.0,30.0


In [293]:
df['value'].sum()

np.float64(1095.0)

In [294]:
df['valueTgt'].sum()

np.float64(1095.0)

In [295]:
df['valueChg'].sum()

np.float64(14.0)

In [296]:
df['percentTgt'].sum()

np.float64(1.0)